<a href="https://colab.research.google.com/github/amgothhrithik/Search_Engine/blob/main/image_search_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

In [ ]:
from datasets import load_dataset

dataset =  load_dataset(
    "ashraq/fashion-product-images-small",
    split="train" )


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/867 [00:00<?, ?B/s]

(…)-00000-of-00002-6cff4c59f91661c3.parquet:   0%|          | 0.00/136M [00:00<?, ?B/s]

(…)-00001-of-00002-bb459e5ac5f01e71.parquet:   0%|          | 0.00/135M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/44072 [00:00<?, ? examples/s]

In [ ]:
print(dataset.features)

{'id': Value(dtype='int64', id=None), 'gender': Value(dtype='string', id=None), 'masterCategory': Value(dtype='string', id=None), 'subCategory': Value(dtype='string', id=None), 'articleType': Value(dtype='string', id=None), 'baseColour': Value(dtype='string', id=None), 'season': Value(dtype='string', id=None), 'year': Value(dtype='float64', id=None), 'usage': Value(dtype='string', id=None), 'productDisplayName': Value(dtype='string', id=None), 'image': Image(mode=None, decode=True, id=None)}


In [ ]:
print(dataset[100])

{'id': 19311, 'gender': 'Men', 'masterCategory': 'Footwear', 'subCategory': 'Shoes', 'articleType': 'Casual Shoes', 'baseColour': 'White', 'season': 'Fall', 'year': 2011.0, 'usage': 'Casual', 'productDisplayName': 'Nike Men Air Rift MTR White Casual Shoe', 'image': <PIL.Image.Image image mode=RGB size=60x80 at 0x7BD768998450>}


In [ ]:


from sentence_transformers import SentenceTransformer, util
from PIL import Image
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
#Load CLIP model
model = SentenceTransformer('clip-ViT-B-32').to(device)

modules.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/4.03k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/604 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
images=dataset['image']

In [ ]:
caption=dataset['productDisplayName']

In [ ]:
embedding=[]
batch_size=32
image_embeddings = model.encode(images, convert_to_tensor=True, batch_size=batch_size, device=device)
caption_embeddings = model.encode(caption, convert_to_tensor=True, batch_size=batch_size, device=device)

# Compute final embedding
final_embeddings = 0.6 * caption_embeddings + 0.4 * image_embeddings


In [ ]:
final_embeddings.shape

torch.Size([44072, 512])

In [ ]:
import numpy as np

np.save("/content/drive/MyDrive/Data/search_engine/embeddings.npy", final_embeddings.cpu().numpy())  # Move to CPU before saving
